# CCAM intake sandbox

In [1]:
import glob
import pathlib
import traceback
from datetime import datetime

import xarray as xr

from ecgtools import Builder
from ecgtools.builder import INVALID_ASSET, TRACEBACK

/g/data/v14/tm4888/miniconda3/envs/intake-env/lib/python3.10/site-packages/fastprogress/fastprogress.py:107: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


In [2]:
import os
import dask.config
from dask.distributed import Client,LocalCluster
from dask_jobqueue import PBSCluster
walltime = '01:00:00'
cores = 48
memory = '192GB'
  
cluster = PBSCluster(walltime=str(walltime), cores=cores, memory=str(memory),processes=cores,
                     job_extra=['-q normal','-P xv83','-l ncpus='+str(cores),'-l mem='+str(memory),
                                '-l storage=gdata/xv83+gdata/v14+gdata/ux62+scratch/xv83+gdata/rt52+gdata/ik11+gdata/cj50+gdata/jk72+gdata/hh5'],
                     local_directory='$TMPDIR',
                     header_skip=["select"])
cluster.scale(jobs=1)

/g/data/v14/tm4888/miniconda3/envs/intake-env/lib/python3.10/site-packages/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/g/data/v14/tm4888/miniconda3/envs/intake-env/lib/python3.10/site-packages/dask_jobqueue/core.py:293: FutureWarning: header_skip has been renamed to job_directives_skip. You are still using it (even if only set to []; please also check config files). If you did not set job_directives_skip yet, header_skip will be respected for now, but it will be removed in a future release. If you already set job_directives_skip, header_skip is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/

In [3]:
cluster

PBSCluster(d1cf24ec, 'tcp://10.6.121.8:41913', workers=48, threads=48, memory=179.04 GiB)

In [7]:
root = '/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/'

In [8]:
files = sorted(glob.glob(root+'*/*/*'))
len(files)

32964

In [9]:
ds = xr.open_dataset(files[0])
ds

<xarray.Dataset>
Dimensions:   (time: 8760, lat: 611, lon: 928, bnds: 2)
Coordinates:
  * lon       (lon) float64 89.3 89.4 89.5 89.6 89.7 ... 181.7 181.8 181.9 182.0
  * lat       (lat) float64 -52.3 -52.2 -52.1 -52.0 -51.9 ... 8.4 8.5 8.6 8.7
  * time      (time) object 1951-01-01 00:00:00 ... 1951-12-31 23:00:00
Dimensions without coordinates: bnds
Data variables:
    CAPE      (time, lat, lon) float32 ...
    lat_bnds  (lat, bnds) float32 ...
    lon_bnds  (lon, bnds) float32 ...
Attributes: (12/19)
    axiom_version:                  0.1.0
    axiom_schemas_version:          0.1.0
    axiom_schema:                   CORDEX
    contact:                        Marcus Thatcher (Marcus.Thatcher@csiro.au)
    Conventions:                    CF-1.7
    CORDEX_domain:                  AUS-10i
    ...                             ...
    model_id:                       CSIRO-CCAM-2203
    product:                        output
    project_id:                     CORDEX
    rcm_version_id:                 v1
    references:                     https://confluence.csiro.au/display/CCAM
    tracking_id:                    a66c2c28-cf6f-45e6-afd8-c9dfe18a10d2

In [11]:
path = pathlib.Path(files[0])
path.stem.split('_')

['CAPE',
 'AUS-10i',
 'NCC-NorESM2-MM',
 'historical',
 'r1i1p1f1',
 'CSIRO-CCAM-2203',
 'v1',
 '1hr',
 '19510101-19511231']

## use this as an error check?

In [12]:
variable_list = [var for var in ds if 'long_name' in ds[var].attrs]
variable_list

['CAPE']

In [15]:
split = path.stem.split('_')

In [16]:
split[0]

'CAPE'

In [ ]:
def parse_CCAM(file):
    """CCAM data stored in"""
    file = pathlib.Path(file)
    info = {}

    try:
        stem = file.stem
        split = stem.split('_')
        variable = split[0]
        domain = split[1]
        host_GCM = split[-2]
        if len(temporal) == 2:
            month_number = int(temporal)
            time_period = 'monthly'
            temporal = datetime(2020, month_number, 1).strftime('%b').upper()

        elif temporal == 'ANN':
            time_period = 'annual'
        else:
            time_period = 'seasonal'

        with xr.open_dataset(file, chunks={}, decode_times=False) as ds:
            variable_list = [var for var in ds if 'long_name' in ds[var].attrs]

            info = {
                'source': source,
                'temporal': temporal,
                'time_period': time_period,
                'variable': variable_list,
                'path': str(file),
            }

        return info

    except Exception:
        return {INVALID_ASSET: file, TRACEBACK: traceback.format_exc()}

In [ ]:
cluster.shutdown()